Hello World,

My name is Alex LordThorsen and I'm going to be talking about Flask and (if we have time) a couple of Flask Plugins.

#Topics:
* Why Flask?
* A Simple Example
* The app Object
* Routes Using The App
* What Do Routes Return?
* The Development Server
* The Debug Stack Traces and Consoles
* An Example Template
* Returning JSON
* The g and session Object
* Request Methods and Metadata
* Configuration
* All Flask Objects


#Why Flask?
Flask is very simple and clean. It amazing easy to create a working application which serves real data (the heart_beat example is production code with 588 lines of code of which 231 is tests and 51 are project configuration that I wrote in 6 hours). Flask also lets you just start writing code without having to deal with any boiler plate (beyond two lines).

[The flaskr example page](https://github.com/mitsuhiko/flask/blob/master/examples/flaskr/flaskr.py) is a fully functional blog (logins, logouts, page addition, homepage) in 110 lines of code backed by a sqlite3 database.

Flask is also just an [amazing code base](https://github.com/mitsuhiko/flask). There has been more then one occuasion where I've looked at the source code to answer a question and it's always a pleasant experience.

The Flask community is also very active and rabid. [The extensions library is filled with Gem's](http://flask.pocoo.org/extensions/) like [Flask-Cache](https://pythonhosted.org/Flask-Cache/), [Flask-DebugToolbar](http://flask.pocoo.org/extensions/), [Flask-lesscss](https://pypi.python.org/pypi/flask-lesscss), [Flask-login](https://pypi.python.org/pypi/Flask-Login), [Flask-WTF](https://pypi.python.org/pypi/Flask-WTF)and [Flask-SQlAlchemy](https://pypi.python.org/pypi/Flask-SQLAlchemy).

If no one has made a Flask Extension that suits your needs then [creating an extension](http://flask.pocoo.org/docs/0.10/extensiondev/) requires about an hour worth of reading, a `setup.py` file, and  a class that modifies a flask app object.

##What Else Is Out There?
[Django has some of the best tutorials on the internet](https://www.djangoproject.com/)

[I've heard many good things about bottle](http://bottlepy.org/docs/dev/index.html)

[Pyramid is a little harder to setup but is also a wonderful project](https://pyramid.readthedocs.org/en/latest/)

#So How Do I Use Flask?

In [22]:
from flask import Flask
app = Flask("the_flask_module")
# The app object is really the heart of flask. All extensions modify the app object and almost
# all of the flask magic comes from calling methods in the app object.


@app.route("/")
def home_page():
    return "I'm a home page"

# If you've never seen decorators (the @app syntax) before then go read this later:
# http://thecodeship.com/patterns/guide-to-python-function-decorators/

# app.run is a blocking call that won't exit unless there's a user interrupt or a system error. 
# You'll have to controll-c to kill this cell to move on to the next cells.
app.run(host="0.0.0.0", port=5000)
# You now have a running web server. go to http://localhost:5000/ in a web browser or curl http://localhost:5000

# Well, That's Cute. Where's Everything Else?
What you see in this example is about 50% of everything there is to Flask. There's some things about sessions, configuration, templating, url routing, http headers, http requests, cookies, and message flashing but this is the majority of everything. To be fair the `@app.route()` call is doing thousands of Python WSGI magic and the `app.run()` call is spinning up a entire http server with a WSGI module soooo...


## By Default There is No Support For
* Databases
* Forms
* An Admin page
* Page indexing
* Asynchronous calls
* Login Support
* File management
* Lots of other defaults in other frameworks.

This is by design.

"The idea of Flask is to build a good foundation for all applications. Everything else is up to you or extensions." -- Armin Ronacher, Author of Flask 

[All of those extra things are found in Flask Extensions which we'll cover in a the flask_extensions_notebook](http://flask.pocoo.org/extensions/)

# So what is a route?
In Flask a route is a function that has a `@app.route()` decorator above it. These define what url's route to what functions and what variables are passed to those functions.

In [36]:
from flask import Flask
app = Flask("the_flask_module")

# Let's look at some of the things that you can do with routes.
@app.route("/hello")
def hello_page():
    return "I'm a hello page"

# You can go up to the HTML URL lenght limit of 2000 characters
# http://stackoverflow.com/questions/417142/what-is-the-maximum-length-of-a-url-in-different-browsers
@app.route("/hello/one_step_deeper")
def hello_deeper():
    return "I'm a hello page"

# the_last_slash_is_optional
@app.route("/hello/optional_slash/")
def optional_extra_slash():
    return "I can have a slash"

# Can't have slash
@app.route("/hello/no_slash")
def no_extra_slash():
    return "No slash for you."

# variables
@app.route('/<text_to_echo>/')
def echo(text_to_echo):
    return "echo {}".format(text_to_echo)

# variables but only ints
@app.route('/test/<int:int_to_echo>/')
def echo_only_ints(int_to_echo):
    return "echo {}".format(int_to_echo)

# the request object is where we can get http header information
from flask import request
@app.route('/', methods=["GET", "POST"])
def get_or_post():
    return request.method

@app.route('/all_methods', methods=["GET", "HEAD", "POST", "PUT", "DELETE", "CONNECT", "OPTIONS", "TRACE"])
def all_methods():
    return request.method


app.run(host="0.0.0.0", port=5000)
# You now have a running web server. go to http://localhost:5000/ in a web browser or curl http://localhost:5000

# Having a two (or more) routes with the same function name name will cause an AssertionError exception.

In [40]:
from flask import Flask
app = Flask("the_flask_module")


@app.route("/hello")
def hello_page():
    return "I'm a hello page"

@app.route("/different_route")
def hello_page():
    return "Different route"


app.run(host="0.0.0.0", port=5000)
# You now have a running web server. go to http://localhost:5000/ in a web browser or curl http://localhost:5000

AssertionError: View function mapping is overwriting an existing endpoint function: hello_page

# Having a two (or more) routes with the same URL path will succeed but the second route will be unreachable.

In [42]:
from flask import Flask
app = Flask("the_flask_module")

# Having a two (or more) routes with the same name will cause an error.
@app.route("/hello")
def hello_page():
    return "I'm a hello page"

@app.route("/hello")
def second_hello_page():
    return "Different route"


app.run(host="0.0.0.0", port=5000)
# You now have a running web server. go to http://localhost:5000/ in a web browser or curl http://localhost:5000

#Flask Has a Top Down Order of Precedence
If you have multiple routes that could be potential matches for a request then the top most route (closest to line 0) will be the route which is mapped to.

In [44]:
from flask import Flask
app = Flask("the_flask_module")

# variables
@app.route('/<text_to_echo>/')
def echo(text_to_echo):
    return "echo {}".format(text_to_echo)

# variables
@app.route('/<text_to_echo>/')
def echo_two(text_to_echo):
    return "Totally going to do my own thing."


app.run(host="0.0.0.0", port=5000)
# You now have a running web server. go to http://localhost:5000/ in a web browser or curl http://localhost:5000

#What do Routes Return?

In [ ]:
import os
from flask import Flask
# This is some path magic that's required because this is an IPython notebook.
template_dir = os.path.join(os.getcwd(), "the_flask_module", "templates")
app = Flask("the_flask_module", template_folder=template_dir)

# We've seen that routes can return straight strings
@app.route('/just_string')
def just_string():
    return "I'm just a poor string!"

from flask import render_template
# Flask also allows you to call a templating engine (which in turn just returns straight strings but prettier)
@app.route('/')
def simple_template():
    return render_template("simple_example.html")

# You can pass arguments to templates which can be tested.
@app.route('/if/')
def if_template():
    return render_template("if_example.html", oxford_comma=True)

# Or arguments that are used to loop.
@app.route('/for/')
def for_template():
    return render_template("for_example.html", word_list=["Nothing", "to", "see", "here."])


app.run(host="0.0.0.0", port=5000)
# You now have a running web server. go to http://localhost:5000/ in a web browser or curl http://localhost:5000

#You Know, I Think JSON is straing data as well...

# Has Anyone Thought About Making API's this way?
[eve](http://python-eve.org/) and [Flask-RESTful](https://flask-restful.readthedocs.org/en/0.3.3/) are two great choices for API design.